# Experiment 3, Model 2

#### Model Setup

Run models in the following order, using their output labels as features for the next model:

1. Multiclass Person Name + Occupation Sequence Classifier

2. Multilabel Document Classifier

Train the first model and then run it over the entire dataset.

***

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/experiment_input/`
    * Prediction Data: Data: under directory `../data/token_clf_data/model_output/experiment3/`
* Word Embeddings
    * Custom fastText (word2vec with subwords) embeddings of 100 dimensions trained on the CRC Archives catalog's descriptive metadata (harvested October 2020)
    
***

**Table of Contents**

[I.](#i) Stereotype + Omission Classifier
* [Preprocessing](#prep)
* [Training & Prediction](#tp)
* [Evaluation](#eval)

[II.](#ii) Predict Over All Data

Load programming resources:

In [1]:
# For custom functions and variables
import utils, utils1, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For classification
import scipy
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

Define resources for the models:

In [2]:
Path(config.experiment_input_path).mkdir(parents=True, exist_ok=True)    # For train, devtest, and blind test data
# Path(config.experiment1_output_path).mkdir(parents=True, exist_ok=True)  # For predictions
# Path(config.experiment1_agmt_path).mkdir(parents=True, exist_ok=True)    # For agreement metrics

# predictions_dir = config.experiment3_path+"5fold/output/"      # For predictions
# Path(predictions_dir).mkdir(parents=True, exist_ok=True)
# agreement_dir = config.experiment3_path+"5fold/agreement/"     # For agreement metrics
# Path(agreement_dir).mkdir(parents=True, exist_ok=True)

predictions_dir = config.experiment3_path+"5fold/with_manual_labels/output/"              # For predictions with features as manual labels
Path(predictions_dir).mkdir(parents=True, exist_ok=True)  
agreement_dir = config.experiment3_path+"5fold/with_manual_labels/agreement/"             # For agreement metrics with features as manual labels
Path(agreement_dir).mkdir(parents=True, exist_ok=True)

In [3]:
# Model 2:
pers_o_label_subset = ["B-Unknown", "I-Unknown", "B-Feminine", "I-Feminine", "B-Masculine", "I-Masculine", "B-Occupation", "I-Occupation"]
# Model 3:
so_label_subset = ["B-Stereotype", "I-Stereotype", "B-Omission", "I-Omission"]

In [4]:
pers_o_label_tags = {
    "Unknown": ["B-Unknown", "I-Unknown"], "Feminine": ["B-Feminine", "I-Feminine"], "Masculine": ["B-Masculine", "I-Masculine"],
     "Occupation": ["B-Occupation", "I-Occupation"]
    }
so_label_tags = {
    "Stereotype": ["B-Stereotype", "I-Stereotype"], "Omission": ["B-Omission", "I-Omission"]
             }

In [5]:
d = 100               # dimensions of word embeddings (should match utils1.py) for file names
target_labels = "so"  # for file names

<a id="i"></a>
## I. Stereotype + Omission Classifier
<a id="a"></a>
### Preprocessing

Load the document classification model's input data:

In [6]:
# # For 60-20-20 split
# train = pd.read_csv(config.docc_path+"model_input/"+"{}_splits_as_csv/aggregated_final_train.csv".format(target_labels), index_col=0)
# dev = pd.read_csv(config.docc_path+"model_input/"+"{}_splits_as_csv/aggregated_final_validate.csv".format(target_labels), index_col=0)
# test = pd.read_csv(config.docc_path+"model_input/"+"{}_splits_as_csv/aggregated_final_test.csv".format(target_labels), index_col=0)
# df_exp = pd.concat([train, dev, test])
# df_exp["label"] = df_exp["label"].fillna("{'None'}")
# df_exp = df_exp.loc[~df_exp.description.isna()]
# df_exp = utils.getColumnValuesAsLists(df_exp, "label")
# # df_exp.head()
# ------------------------------
# For modified 5-fold cross validation
df = pd.read_csv(config.tokc_path+"experiment_input/document_5fold.csv", index_col=0)
df_exp = utils.getColumnValuesAsLists(df, "label")
df_exp = df_exp.drop(columns=["subset"])
df_exp.head()

,description_id,start_offset,end_offset,field,description,label,fold
0,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",[Omission],split3
1,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,[],split2
2,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,[],split0
3,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,"[Omission, Stereotype]",split0
4,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,[Omission],split3


Load the Linguistic labels as features and associate description IDs to the data, creating one row per description ID:

In [7]:
feat1_col = "label_pers_o_expected"  #"label_pers_o_predicted""

In [24]:
# features_filename = "crf_{a}_{t}_baseline_fastText{d}_predictions_ALLDATA.csv".format(a="arow", t="pers_o", d=d)
# ------------------------------
features_filename = "crf_{a}_baseline_fastText{d}_{c}_nolingfeatures_loose_evaluation.csv".format(a="arow", d=d, c="pers_o")
df_features = pd.read_csv(config.experiment3_path+"5fold/output"+features_filename, usecols=["sentence_id", "token_id", feat1_col])
df_features = utils1.getColumnValuesAsLists(df_features, feat1_col)
# df_features = utils.getColumnValuesAsLists(df_features, "label_pers_o_expected")
df_features.head()

,sentence_id,token_id,label_pers_o_expected
0,8,233,['Masculine']
1,8,234,['Masculine']
2,8,235,['O']
3,8,236,['O']
4,8,237,['O']


In [25]:
df_desc = pd.read_csv(config.agg_path+"descs_sents_tokens_anns.csv", usecols=["description_id", "sentence_id", "token_id"])
df_desc = df_desc.set_index("description_id")
df_desc = utils1.getColumnValuesAsLists(df_desc, "sentence_id")
df_desc = utils1.getColumnValuesAsLists(df_desc, "token_id")
df_desc_exploded = df_desc.explode(["sentence_id", "token_id"])
df_desc_exploded = df_desc_exploded.reset_index()
df_desc_exploded = df_desc_exploded.astype("int64")
# df_desc_exploded.head()

In [26]:
joined = df_features.join(df_desc_exploded.set_index(["sentence_id", "token_id"]), on=["sentence_id", "token_id"])
grouped = utils.implodeDataFrame(joined, ["description_id"]).reset_index()
# grouped.head()

Flatten the lists of values in the `pers_o_expected` column and remove duplicates from the lists:

In [31]:
perso = utils1.flattenFeatureCol(grouped, feat1_col)
# print(perso[:3])
old_perso = [[label.replace("'","") for label in labels] for labels in perso]
perso = []
for labels in old_perso:
    if (len(labels) > 1) and ("O" in labels):
        labels.remove("O")
    perso += [labels]
print(perso[:3])  # Looks good

[['O'], ['Masculine', 'Unknown'], ['O']]


In [32]:
grouped.insert(len(grouped.columns), "doc_"+feat1_col, perso)
grouped.head()

,description_id,sentence_id,token_id,label_pers_o_expected,doc_label_pers_o_expected
0,0,"[0, 0, 0]","[0, 1, 2]","[['O'], ['O'], ['O']]",[O]
1,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]","[['O'], ['O'], ['O'], ['O'], ['O', 'Unknown', ...","[Masculine, Unknown]"
2,2,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2...","[['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O...",[O]
3,3,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[233, 234, 235, 236, 237, 238, 239, 240, 241, ...","[['Masculine'], ['Masculine'], ['O'], ['O'], [...","[Occupation, Masculine]"
4,4,"[11, 11, 11]","[308, 309, 310]","[['O'], ['O'], ['O']]",[O]


Join the Person Name + Occupation feature column to the document classification model data:

In [37]:
features = grouped[["description_id", "doc_"+feat1_col]]
join_on = "description_id"
df = df_exp.join(features.set_index(join_on), on=join_on)
df = df.loc[~df.description.isna()]
df.head()

,description_id,start_offset,end_offset,field,description,label,fold,doc_label_pers_o_expected
0,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",[Omission],split3,"[Masculine, Unknown]"
1,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,[],split2,[Masculine]
2,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,[],split0,"[Occupation, Unknown]"
3,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,"[Omission, Stereotype]",split0,"[Feminine, Occupation, Masculine, Unknown]"
4,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,[Omission],split3,"[Masculine, Unknown]"


In [41]:
# df.label.value_counts()              # Looks good
# df["doc_"+feat1_col].value_counts()  # Looks good

Make sure the Person Name + Occupation labels are sets (each row's list should not have any repeated values), and that "O" is only assigned to a description if it has no other labels:

In [39]:
# perso_labels = list(df[feat1_col])
# unique_perso_labels = [list(set(label_list)) for label_list in perso_labels]
# final_perso_labels = []
# for labels in unique_perso_labels:
#     if (len(labels) > 1) and ("O" in labels):
#         labels.remove("O")
#     final_perso_labels += [labels]
# print(final_perso_labels[:3])

In [50]:
# df[feat1_col] = final_perso_labels
# df.head()

,description_id,start_offset,end_offset,field,description,label,fold,label_pers_o_predicted
0,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",[Omission],split3,[Unknown]
1,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,[],split2,[Unknown]
2,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,[],split0,"[Unknown, Occupation, Masculine]"
3,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,"[Omission, Stereotype]",split0,"[Masculine, Feminine, Unknown, Occupation]"
4,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,[Omission],split3,[Unknown]


Define the train (80% of the data) and test (20% of the data) splits:

In [42]:
split_col = "fold"
splits = df[split_col].unique()
splits.sort()
print(splits)
train0, test0 = list(splits[:4]), splits[4]
train1, test1 = list(splits[1:]), splits[0]
train2, test2 = list(splits[2:])+[splits[0]], splits[1]
train3, test3 = list(splits[3:])+list(splits[:2]), splits[2]
train4, test4 = [splits[4]]+list(splits[:3]), splits[3]
runs = [(train0, test0), (train1, test1), (train2, test2), (train3, test3), (train4, test4)]
for run in runs:
    print(run)

['split0' 'split1' 'split2' 'split3' 'split4']
(['split0', 'split1', 'split2', 'split3'], 'split4')
(['split1', 'split2', 'split3', 'split4'], 'split0')
(['split2', 'split3', 'split4', 'split0'], 'split1')
(['split3', 'split4', 'split0', 'split1'], 'split2')
(['split4', 'split0', 'split1', 'split2'], 'split3')


In [43]:
def binarizeMultilabelTrainColumn(df_col):
    mlb = MultiLabelBinarizer()
    binarized = mlb.fit_transform(df_col)
    return mlb, binarized

def binarizeMultilabelDevColumn(mlb, df_col):
    binarized = mlb.transform(df_col)
    return binarized

Vectorize the documents, and binarize the features and targets:

In [15]:
# train_df = df.loc[df.subset == "train"]
# dev_df = df.loc[df.subset == "dev"]
# target_col = "label"
# feat1_col = "doc_ling_pred"

In [16]:
# mlb_target, y_train = binarizeMultilabelTrainColumn(train_df["label"])
# y_dev = binarizeMultilabelDevColumn(mlb_target, dev_df["label"])
# print(y_train.shape, y_dev.shape)

(16397, 3) (5452, 3)


In [18]:
# mlb_feat1, train_feat1 = binarizeMultilabelTrainColumn(train_df[feat1_col])
# dev_feat1 = binarizeMultilabelDevColumn(mlb_feat1, dev_df[feat1_col])
# print(train_feat1.shape, dev_feat1.shape)

(16397, 3) (5452, 3)


In [19]:
# cvectorizer = CountVectorizer()
# tfidf = TfidfTransformer()
# train_docs = cvectorizer.fit_transform(train_df["description"])
# dev_docs = cvectorizer.transform(dev_df["description"])
# train_docs = tfidf.fit_transform(train_docs)
# dev_docs = tfidf.transform(dev_docs)
# print(train_docs.shape, dev_docs.shape)

(16397, 26960) (5452, 26960)


In [20]:
# train_feats = scipy.sparse.csr_matrix(train_feat1)
# dev_feats = scipy.sparse.csr_matrix(dev_feat1)

Concatenate the documents and features, creating one scipy sparse matrix for the train data and another for the dev data:

In [21]:
# X_train = scipy.sparse.hstack([train_docs, train_feats])
# X_dev = scipy.sparse.hstack([dev_docs, dev_feats])
# print(X_train.shape, X_dev.shape)

(16397, 26963) (5452, 26963)


<a id="tp"></a>
### Train & Predict

In [46]:
a = "sgd-svm"

In [48]:
pred_df = pd.DataFrame()
target_col = "label"
feat1_col = "doc_"+feat1_col
for run in runs:
    # Get the train (80%) and test (20%) subsets of data
    train_splits, test_split = run[0], run[1]
    print("Training on:", train_splits)
    train_df = df.loc[df[split_col].isin(train_splits)]
    dev_df = df.loc[df[split_col] == test_split]
    
    # Binarize the features
    mlb_feat, train_feat = binarizeMultilabelTrainColumn(train_df[feat1_col])
    dev_feat = binarizeMultilabelDevColumn(mlb_feat, dev_df[feat1_col])
    
    # Vectorize the documents (descriptions)
    cvectorizer = CountVectorizer()
    tfidf = TfidfTransformer()
    train_docs = cvectorizer.fit_transform(train_df["description"])
    dev_docs = cvectorizer.transform(dev_df["description"])
    train_docs = tfidf.fit_transform(train_docs)
    dev_docs = tfidf.transform(dev_docs)
    
    # Concatenate the features and documents
    X_train = scipy.sparse.hstack([train_docs, train_feat])
    X_dev = scipy.sparse.hstack([dev_docs, dev_feat])
    
    # Binarize targets
    mlb_target, y_train = binarizeMultilabelTrainColumn(train_df["label"])
    y_dev = binarizeMultilabelDevColumn(mlb_target, dev_df["label"])

    # Train a classification model
    clf = OneVsRestClassifier(SGDClassifier(loss="hinge"))  # Support Vector Machines loss function
    clf.fit(X_train, y_train)
    
    # Predict with the trained model
    print("Predicting on:", test_split)
    predictions = clf.predict(X_dev)
    pred_labels = mlb_target.inverse_transform(predictions)    
    if pred_df.shape[0] > 0:
        next_pred_df = dev_df.copy()
        next_pred_df.insert(len(next_pred_df.columns), "{}_label".format(a), pred_labels)
        pred_df = pd.concat([pred_df, next_pred_df])
    else:
        pred_df = dev_df.copy()
        pred_df.insert(len(pred_df.columns), "{}_label".format(a), pred_labels)

print("Modified 5-fold cross-validation complete!")
print(pred_df.shape)

Training on: ['split0', 'split1', 'split2', 'split3']
Predicting on: split4
Training on: ['split1', 'split2', 'split3', 'split4']
Predicting on: split0
Training on: ['split2', 'split3', 'split4', 'split0']
Predicting on: split1
Training on: ['split3', 'split4', 'split0', 'split1']
Predicting on: split2
Training on: ['split4', 'split0', 'split1', 'split2']
Predicting on: split3
Modified 5-fold cross-validation complete!
(27312, 9)


In [49]:
pred_df = pred_df.rename(columns={"label":"manual_label"})
pred_df.head()

,description_id,start_offset,end_offset,field,description,manual_label,fold,doc_label_pers_o_expected,sgd-svm_label
6,3027,627,1162,Biographical / Historical,Thomas Young was probably born in 1725. By the...,[Stereotype],split4,"[Occupation, Masculine, Unknown]","(Stereotype,)"
7,3397,8095,8334,Biographical / Historical,Andrew Tait worked on Paramecium in Beale's la...,[Omission],split4,"[Occupation, Masculine, Unknown]","(Omission,)"
10,4736,9951,10026,Biographical / Historical,Delivered by Thomson to teachers in Darlington.,[Omission],split4,"[Occupation, Masculine, Unknown]","(Omission,)"
14,4712,4199,4485,Biographical / Historical,"This was gifted by Thomson to his secretary, M...",[Omission],split4,"[Feminine, Occupation, Masculine, Unknown]","(Omission,)"
22,15684,845,1179,Biographical / Historical,Catherine Robina Borland was responsible for t...,[],split4,[O],"(,)"


Save the predictions data:

In [50]:
pred_df.to_csv(predictions_dir+"aggregated_final_validate_predictions_docclf_{a}_{t}.csv".format(a=a, t=target_labels))

Build a pipeline:

In [23]:
# doc_clf = Pipeline([
#     ("clf", OneVsRestClassifier(SGDClassifier(loss="hinge")))  # Support Vector Machines loss function
# ])

In [24]:
# doc_clf.fit(X_train, y_train)
# predictions = doc_clf.predict(X_dev)

### Peformance

Calculate performance metrics for the Stochastic Gradient Descent classifier

In [25]:
# print("Dev Test Accuracy:", np.mean(predictions == y_dev))

Dev Test Accuracy: 0.9388603570555148


In [51]:
classes = clf.classes_  #doc_clf.classes_
print(classes)
original_classes = mlb_target.classes_
print(original_classes)
label_dict = dict(zip(original_classes, classes))

[0 1 2]
['' 'Omission' 'Stereotype']


Create a [confusion matrix](https://scikit-learn.org/stable/modules/model_evaluation.html#multilabel-confusion-matrix) of the results, where, for class *i*:
* Count of true negatives (TN) is at position *i*,0,0
* Count of false negatives (FN) is at position *i*,1,0
* Count of true positives (FP) is at position *i*,1,1
* Count of false positives (PF) is at position *i*,0,1

In [52]:
y_dev = binarizeMultilabelDevColumn(mlb_target, pred_df["manual_label"])
predictions = mlb_target.transform(pred_df["{}_label".format(a)])
assert len(y_dev) == len(predictions)

In [53]:
matrix = multilabel_confusion_matrix(y_dev, predictions, labels=classes)

In [54]:
scores = utils.getPerformanceMetrics(y_dev, predictions, matrix, classes, original_classes, label_dict)
scores = scores.tail(2) # Remove row for 'None'
scores = scores.drop(columns="true_neg")  # Not accurate because considers 'None' a class
scores["labels"] = original_classes[1:]
scores

,labels,false_neg,true_pos,false_pos,precision,recall,f_1
1,Omission,1698,2334,430,0.844428,0.578869,0.686875
2,Stereotype,390,1211,73,0.943146,0.756402,0.839515


Save the performance results:

In [55]:
scores.to_csv(agreement_dir+"docclf_{a}_{t}_baseline_performance.csv".format(a=a, t=target_labels))

Add the predicted labels to the dev data:

In [31]:
# pred_dev_labels = mlb_target.inverse_transform(predicted_dev)
# # pred_dev_labels[0]

Add the classifier's labels to the `aggregated_validate.csv` DataFrame of descriptions to facilitate error analysis:

In [62]:
# dev_df = dev_df.rename(columns={"label":"manual_label"})
# dev_df.insert(len(dev_df.columns), "{a}_label".format(a=a), pred_dev_labels)
# dev_df.head()
# # print(len(pred_dev_labels), dev_df.shape)

Save this version of the data:

In [33]:
# dev_df.to_csv(dir_path+"aggregated_final_validate_predictions_docclf_{a}_{t}.csv".format(a=a, t=target_labels))

***

#### *For train-dev-test (i.e., 60-20-20) approach*

<a id="ii"></a>
## II. Predict Over All Data

### Preprocessing
Vectorize the documents, and binarize the features and targets:

In [34]:
target_col = "label"
feat1_col = "doc_ling_pred"

In [35]:
y_all = binarizeMultilabelDevColumn(mlb_target, df["label"])
print(y_all.shape)

(27312, 3)


In [36]:
all_feat1 = binarizeMultilabelDevColumn(mlb_feat1, df[feat1_col])
print(all_feat1.shape)

(27312, 3)


In [37]:
all_docs = cvectorizer.transform(df["description"])
all_docs = tfidf.transform(all_docs)
print(all_docs.shape)

(27312, 26960)


In [38]:
all_feats = scipy.sparse.csr_matrix(all_feat1)

Concatenate the documents and features, creating one scipy sparse matrix for the train data and another for the dev data:

In [39]:
X_all = scipy.sparse.hstack([all_docs, all_feats])
print(X_all.shape)

(27312, 26963)


### Predict

In [40]:
predicted_all = doc_clf.predict(X_all)

### Peformance

Calculate performance metrics for the Stochastic Gradient Descent classifier

In [41]:
print("Accuracy:", np.mean(predicted_all == y_all))

Accuracy: 0.9497046475297793


In [42]:
matrix = multilabel_confusion_matrix(y_all, predicted_all, labels=classes)

In [43]:
scores = utils.getPerformanceMetrics(y_all, predicted_all, matrix, classes, original_classes, label_dict)
scores = scores.tail(2) # Remove row for 'None'
scores = scores.drop(columns="true_neg")  # Not accurate because considers 'None' a class
scores["labels"] = original_classes[1:]
scores

,labels,false_neg,true_pos,false_pos,precision,recall,f_1
1,Omission,1637,2395,252,0.904798,0.593998,0.717173
2,Stereotype,343,1258,48,0.963247,0.785759,0.865497


Save the performance results:

In [44]:
# dir_path = config.tokc_path+"/experiment2/output/"
scores.to_csv(dir_path+"docclf_{a}_{t}_baseline_performance_ALLDATA.csv".format(a=a, t=target_labels))

Add the predicted labels to the dev data:

In [45]:
pred_all_labels = mlb_target.inverse_transform(predicted_all)

Add the classifier's labels to the `aggregated_validate.csv` DataFrame of descriptions to facilitate error analysis:

In [46]:
df = df.rename(columns={"label":"manual_label"})
df.insert(len(df.columns), "{a}_label".format(a=a), pred_all_labels)
df.head()
# print(len(pred_all_labels), df.shape)

,description_id,start_offset,end_offset,field,description,subset,manual_label,doc_ling_pred,sgd-svm_label
4699,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",train,[Omission],[Gendered-Role],"(None,)"
8942,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,train,[None],[Gendered-Pronoun],"(None,)"
5440,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,train,[None],[],"(None,)"
3474,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,train,"[Omission, Stereotype]","[Generalization, Gendered-Role]","(Omission, Stereotype)"
4769,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,train,[Omission],[Gendered-Pronoun],"(Omission,)"


Save this version of the data:

In [47]:
df.to_csv(dir_path+"aggregated_final_validate_predictions_docclf_{a}_{t}_ALLDATA.csv".format(a=a, t=target_labels))